Line/Space Function

In [217]:
# Import the needed extensions
import math
import klayout.db as db

def LS_cell(tone:str="D",size:float=0.100,pitch:float=0.500,cell_size:float=25,angle:float=0,x2y:float=1,metro_structure:bool = True):

#### Setup ####

    #Initial tone check
    if tone == "D":
            tone = "D"
    elif tone == "C":
            tone = "C"
    else: return TypeError("Error: Tone must be (D)ark or (C)lear)")

    #Create the layout
    layout = db.Layout()
    layout.dbu = 0.001 #um

    #Define initial layers
    l_line = layout.layer(1,0) #line layer, sacrificial

    #Check pitch of the LS array
    pitch_check = size/pitch
    iso = pitch_check < 0.1 #boolean
    bar3 = pitch_check > 0.75 #also boolean

    #Determine naming
    if iso:
        pitch_type = 'iso'
    elif bar3:
        pitch_type = '3bar'
    else:
        pitch_type = f"{pitch}umPitch"

    #Conver angle from degrees to radians
    rad_angle = angle * (math.pi)/(180)


#### Cell creation and shape assignment ####

    #Create the topcell and subsidiary cells
    TopCell = layout.create_cell(f"Initial_cell")
    LineCell = layout.create_cell(f"{size}um_line")

    #Define line dimensions (top and bottom 2x to provide extra length for angled cells)
    l_left = -size/2
    l_right = size/2
    l_2bottom = -cell_size
    l_2top = cell_size

    #Create the main feature shape and insert it into the LineCell cell
    LineCell.shapes(l_line).insert(db.DBox(l_left, l_2bottom, l_right, l_2top))

    #Creates formatting regions
    CellBox = db.DBox((-cell_size/2),-cell_size/2,(cell_size/2),cell_size/2)
    BigBox = db.DBox(-cell_size*1000,-cell_size*1000,cell_size*1000,cell_size*1000)
    BigBox_region = db.Region(BigBox)
    Bar3Line = db.DBox(-1000*1.5*size,-cell_size*1000,1000*1.5*size,cell_size*1000)
    Bar3Line_region = db.Region(Bar3Line)
    DenseLine = db.DBox(-1000*pitch/2,-1000*cell_size,1000*pitch/2,1000*cell_size)
    DenseLine_region = db.Region(DenseLine)


#### Generate the Cell ####

    if iso:
        #Checks tone and adjusts if needed, then instances a single LineCell into the TopCell
        if tone == "C":
            iso_region = db.Region(LineCell.shapes(l_line))
            clear_iso = iso_region ^ BigBox_region
            LineCell.clear_shapes()
            LineCell.shapes(l_line).insert(clear_iso)       
        ls_iso = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(ls_iso)

    elif bar3:
        #Create the Bar3Cell
        Bar3Cell =  layout.create_cell(f"{size}um_3bar_structure")
        Bar3Cell.shapes(l_line).insert(db.DBox(-cell_size,-cell_size,3*l_left,cell_size))
        Bar3Cell.shapes(l_line).insert(db.DBox(3*l_right,-cell_size,cell_size,cell_size))
        
        #Checks tone and adjusts if needed, then instances a single LineCell into the TopCell
        if tone == "C":
            bar3_region = db.Region(LineCell.shapes(l_line))
            clear_bar3 = bar3_region ^ Bar3Line_region
            LineCell.clear_shapes()
            LineCell.shapes(l_line).insert(clear_bar3)
            Bar3Cell.clear_shapes()
        
        #Add all into the TopCell
        ls_3bar_line = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(ls_3bar_line)    
        ls_3bar_edge = db.DCellInstArray(Bar3Cell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(ls_3bar_edge)

    else:
        #Checks tone and adjusts if needed, then instances a single LineCell into the TopCell
        if tone == "C":
            dense_region = db.Region(LineCell.shapes(l_line))
            clear_dense = dense_region ^ DenseLine_region
            LineCell.clear_shapes()
            LineCell.shapes(l_line).insert(clear_dense)
        
        #Instance a left and right array of the LineCell to span the TopCell region. Trigonometry used to calculate step sizes to preserve pitch for angled arrays.
        ls_array_right = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0),db.DVector((pitch*math.cos(rad_angle)),-pitch*math.sin(rad_angle)),db.DVector(0,0),math.ceil(cell_size/pitch),0)
        ls_array_left = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0),db.DVector((-pitch*math.cos(rad_angle)),pitch*math.sin(rad_angle)),db.DVector(0,0),math.ceil(cell_size/pitch),0)
        TopCell.insert(ls_array_right)
        TopCell.insert(ls_array_left)


#### Add metro structures if applicable ####
    metro_spacing = 8 #um

    if metro_structure:
        MetroCell = layout.create_cell(f"{size}um_line_metro_structure")
        MetroShape = db.DBox(-size/2,-size/2,size/2,size/2)
        MetroCell.shapes(l_line).insert(MetroShape)

        if tone == "C":
             metro_insert = db.DCellInstArray(MetroCell,db.DTrans(db.DTrans.M0,-metro_spacing*math.sin(rad_angle),-metro_spacing*math.cos(rad_angle)),db.DVector(2*metro_spacing*math.sin(rad_angle),2*metro_spacing*math.cos(rad_angle)),db.DVector(0,0),2,0)
        else:
             sqrt_calc = math.sqrt(((size)**2)+(metro_spacing**2))
             extra_angle = -size/metro_spacing
             metro_insert = db.DCellInstArray(MetroCell,db.DTrans(db.DTrans.M0,sqrt_calc*math.sin(rad_angle+extra_angle),sqrt_calc*math.cos(rad_angle+extra_angle)),db.DVector(-2*metro_spacing*math.sin(rad_angle),-2*metro_spacing*math.cos(rad_angle)),db.DVector(2*size*math.cos(rad_angle),-2*size*math.sin(rad_angle)),2,2)

        TopCell.insert(metro_insert)


#### Angle transformations, sliver removal, and output ####
    
    #Apply angle rotation for all cells
    t = db.ICplxTrans(1,angle,0,0,0)
    [layout.cell(i).transform(t) for i in TopCell.each_child_cell()]

    #Clip a new cell that covers just the extents of the defined cell size, and eliminate subcells that contain slivers
    output_cell = layout.clip(TopCell,CellBox)
    listicle = []
    size_check = db.DBox(-size/2,-size/2,size/2,size/2)
    [listicle.append(j) for j in output_cell.each_child_cell() if layout.cell(j).dbbox(l_line).width()<size_check.bbox().width() or layout.cell(j).dbbox(l_line).height()<size_check.bbox().height()]
    [layout.cell(k).prune_cell() for k in listicle]

    #Rename new cell, prune old cell
    if metro_structure:
        output_cell.name = (f"LS_Array_{tone}_{size}umSize_{pitch_type}_{angle}degrees_w_metro")
    else:
        output_cell.name = (f"LS_Array_{tone}_{size}umSize_{pitch_type}_{angle}degrees")
    
    TopCell.prune_cell()

    #Export GDS
    layout.write("LS_Tester.gds")

    return output_cell

LS_cell()

Contact Function

In [202]:
# Import the needed extensions
import math
import klayout.db as db

def contact_cell(tone:str="D",size:float=0.100,pitch:float=0.100,cell_size:float=25,angle:float=60,x2y:float=2,metro_structure:bool = True):

#NOTE: Angled mode is currently configured up to 180 degrees, but metro structures do not currently work for angles besides 0 degrees!

#### Setup ####

    #Initial tone check
    if tone == "D":
            tone = "D"
    elif tone == "C":
            tone = "C"
    else: return TypeError("Error: Tone must be (D)ark or (C)lear)")

    #Create the layout
    layout = db.Layout()
    layout.dbu = 0.001 #um

    #Define initial layers
    l_cont = layout.layer(1,0) #line layer, sacrificial

    #Check pitch of the contact array
    pitch_check = size/pitch
    iso = pitch_check < 0.1 #boolean
    donut = pitch_check > 0.75 #also boolean

    #Determine X and Y pitches, based on x2y value (applies to X-pitch only, Y-pitch is untouched)
    xPitch = (size*x2y) + (pitch-size)
    yPitch = pitch
    xSize = (size*x2y)
    ySize = size

    #Determine naming
    if iso:
        pitch_type = 'iso'
    elif donut:
        pitch_type = 'donut'
    else:
        pitch_type = f"{pitch}umPitch"

    #Convert angle from degrees to radians
    rad_angle = angle * (math.pi)/(180)


#### Cell creation and shape assignment ####

    #Create the topcell and subsidiary cells
    TopCell = layout.create_cell(f"Initial_cell")
    ContCell = layout.create_cell(f"{size}um_{x2y}to1_contact")

    #Define contact dimensions
    c_left = -xSize/2
    c_right = xSize/2
    c_bottom = -ySize/2
    c_top = ySize/2

    #Create the main feature shape and insert it into the ContCell
    cont_iso = db.DBox(c_left, c_bottom, c_right, c_top)
    ContCell.shapes(l_cont).insert(cont_iso)

    #Creates formatting regions for later use
    CellBox = db.DBox((-cell_size/2),-cell_size/2,(cell_size/2),cell_size/2)
    BigBox = db.DBox(-cell_size*1000,-cell_size*1000,cell_size*1000,cell_size*1000)
    BigBox_region = db.Region(BigBox)
    LittleDonut = db.DBox(-1500*(xSize),-1500*(ySize),1500*(xSize),1500*(ySize))
    LittleDonut_region = db.Region(LittleDonut)
    DonutBox = BigBox_region-LittleDonut_region
    DonutBox.break_(4,1)
    DenseCont = db.DBox(-1000*xPitch/2,-1000*yPitch/2,1000*xPitch/2,1000*yPitch/2)
    DenseCont_region = db.Region(DenseCont)


#### Generate the Cell ####

    if iso:
        #Checks tone and adjusts if needed, then instances a single cell into the TopCell
        if tone == "C":
            iso_region = db.Region(ContCell.shapes(l_cont))
            clear_iso = BigBox_region-iso_region
            clear_iso.break_(4,1)
            ContCell.clear_shapes()
            ContCell.shapes(l_cont).insert(clear_iso)       
        cont_iso = db.DCellInstArray(ContCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(cont_iso)

    elif donut:
        #Create the DonutCell
        DonutCell = layout.create_cell(f"{size}um_donut_structure")
        DonutCell.shapes(l_cont).insert(DonutBox)
        
        #Checks tone and adjusts if needed
        if tone == "C":
            donut_region = db.Region(ContCell.shapes(l_cont))
            little_donut = donut_region ^ LittleDonut_region
            ContCell.clear_shapes()
            ContCell.shapes(l_cont).insert(little_donut)
            DonutCell.clear_shapes()
        
        #Add all into the TopCell
        cont_donut_center = db.DCellInstArray(ContCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(cont_donut_center)    
        cont_donut_ring = db.DCellInstArray(DonutCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(cont_donut_ring)

    else:
        #Checks tone and adjusts if needed
        if tone == "C":
            dense_region = db.Region(ContCell.shapes(l_cont))
            clear_dense = dense_region ^ DenseCont_region
            clear_dense.break_(4,1)
            ContCell.clear_shapes()
            ContCell.shapes(l_cont).insert(clear_dense)
        
        #Instances the contact cell to span the TopCell region. Trigonometry used to calculate step sizes to preserve pitch for angled arrays.
        #NOTE: This "functions" in its current form, but needs further adjusting, especially for holes, to get a good array of angled contacts
        
        if angle <= 90:
            x_origin = -cell_size+cell_size*2*math.sin(rad_angle)*math.cos(rad_angle)
            y_origin = -cell_size-cell_size*math.sin(rad_angle)*math.cos(rad_angle)
        elif angle <=180:
            x_origin = cell_size-cell_size*math.sin(rad_angle)*math.cos(rad_angle)
            y_origin = -cell_size-cell_size*2*math.cos(rad_angle)*math.sin(rad_angle)
        else:
            return ValueError("Choose an angle between 0 and 180")
        cont_array = db.DCellInstArray(ContCell,db.DTrans(db.DTrans.M0,x_origin,y_origin),
                        db.DVector(xPitch*math.cos(rad_angle),xPitch*math.sin(rad_angle)),db.DVector(-yPitch*math.sin(rad_angle),yPitch*math.cos(rad_angle)),
                        2*math.ceil(cell_size/xPitch),2*math.ceil(cell_size/yPitch))
        TopCell.insert(cont_array)

    print(-xPitch*math.ceil(cell_size/xPitch))
    print(-yPitch*math.ceil(cell_size/yPitch))

#### Add metro structures if applicable ####
    metro_spacing = 8 #um

    if metro_structure:
        MetroCell = layout.create_cell(f"{size}um_line_metro_structure")
        if iso:
             MetroShape = db.DPolygon([db.DPoint(-0.5,0),db.DPoint(0,0.5),db.DPoint(0.5,0),db.DPoint(0,-0.5)])
             MetroCell.shapes(l_cont).insert(MetroShape)
             metro_insert = db.DCellInstArray(MetroCell,db.DTrans(db.DTrans.M0,metro_spacing*math.sin(rad_angle),-metro_spacing*math.cos(rad_angle)),db.DVector(-2*metro_spacing*math.sin(rad_angle),2*metro_spacing*math.cos(rad_angle)),db.DVector(0,0),2,0)

             if tone == "C":
                  MetroShape1 = (db.DPolygon([db.DPoint(-500,metro_spacing*1000),db.DPoint(0,500+metro_spacing*1000),db.DPoint(500,metro_spacing*1000),db.DPoint(0,1000*metro_spacing-500)]))
                  MetroShape2 = (db.DPolygon([db.DPoint(-500,-metro_spacing*1000),db.DPoint(0,500-metro_spacing*1000),db.DPoint(500,-metro_spacing*1000),db.DPoint(0,-1000*metro_spacing-500)]))
                  MShape1_Reg = db.Region(MetroShape1)
                  MShape2_Reg = db.Region(MetroShape2)
                  clear_iso = BigBox_region-iso_region
                  new_clear_cont_temp = clear_iso - MShape1_Reg
                  new_clear_cont = new_clear_cont_temp-MShape2_Reg
                  new_clear_cont.break_(5,2)
                  MetroCell.prune_cell()
                  ContCell.shapes(l_cont).clear()
                  ContCell.shapes(l_cont).insert(new_clear_cont)
             else:
                 TopCell.insert(metro_insert)

        
        elif not donut:
             if angle>0:
                 print("No metro structure for angled contact array at this time")
             else:
                if tone == "D":
                    Dense_Metro = 1000*db.DBox((-2*xPitch)+xSize/2,(-2*yPitch)-size/2,(-xPitch)-xSize/2,(-2*yPitch)+size/2)
                    Dense_Metro_Region = db.Region(Dense_Metro)
                    TopCell.shapes(l_cont).insert(Dense_Metro_Region)
                else:
                    Dense_Metro = 1000*db.DBox((-2*xPitch)-xSize/2,(-2*yPitch)-size/2,(-2*xPitch)+xSize/2,(-2*yPitch)+size/2)
                    Dense_Metro_Region = db.Region(Dense_Metro)
                    TopCell.shapes(l_cont).insert(Dense_Metro_Region)


             MetroCell.prune_cell()




#### Angle transformations, sliver removal, and output ####


    #Apply angle rotation for all cells
    t = db.ICplxTrans(1,-angle,False,0,0)
    [layout.cell(i).transform(t) for i in TopCell.each_child_cell()]

    #Clip a new cell that covers just the extents of the defined cell size, and eliminate subcells that contain slivers
    output_cell = layout.clip(TopCell,CellBox)
    listicle = []
    size_check = db.DBox(-size/2,-size/2,size/2,size/2)
    [listicle.append(j) for j in output_cell.each_child_cell() if layout.cell(j).dbbox(l_cont).width()<size_check.bbox().width() or layout.cell(j).dbbox(l_cont).height()<size_check.bbox().height()]
    [layout.cell(k).prune_cell() for k in listicle]

    #Rename new cell, prune old cell
    if metro_structure:
        output_cell.name = (f"Cont_Array_{tone}_{size}umSize_{pitch_type}_{x2y}to1_x2y_{angle}degrees_w_metro")
    else:
        output_cell.name = (f"Cont_Array_{tone}_{size}umSize_{pitch_type}_{x2y}to1_x2y_{angle}degrees")
    
    TopCell.prune_cell()

    #Export GDS
    layout.write("Cont_Tester.gds")

    return output_cell

contact_cell()

-25.0
-25.0


Line with SRAF function

In [288]:
# Import the needed extensions
import math
import klayout.db as db

def SRAF_cell(tone:str="C",size:float=0.300,pitch:float=0.500,cell_size:float=25,angle:float=0,sraf_size:float=0.05,sraf_step:float=0.2,sraf_num:int=4,metro_structure:bool = True):

#NOTE:No angle support currently, No Metro structures needed

#### Setup ####

    #Initial tone check
    if tone == "D":
            tone = "D"
    elif tone == "C":
            tone = "C"
    else: return TypeError("Error: Tone must be (D)ark or (C)lear)")

    #Create the layout
    layout = db.Layout()
    layout.dbu = 0.001 #um

    #Define initial layers
    l_line = layout.layer(1,0)

    #Check pitches, and set pitch controls for future reference
    pitch_check = size/pitch
    iso = pitch_check < 0.1 #boolean
    too_dense = pitch_check >= 0.1 #also boolean

    sraf_pitch_check = sraf_step/sraf_size
    sraf_nok = sraf_pitch_check<=2

    if sraf_nok:
         sraf_step = 3*sraf_size
         print("SRAF step size too small. Defaulting to 3x sraf size for step size")

    sraf_spacing = sraf_num*(sraf_size+sraf_step)

    if too_dense:
        pitch = 3*(size/2 + sraf_spacing)
        num_lines = math.floor(cell_size/pitch)

    #Determine naming
    if iso:
        pitch_type = 'iso'
    else:
        pitch_type = 'dense'

    #Convert angle from degrees to radians
    rad_angle = angle * (math.pi)/(180)


#### Cell creation and shape assignment ####

    #Create the topcell and subsidiary cells
    TopCell = layout.create_cell(f"Initial_cell")
    LineCell = layout.create_cell(f"{size}um_line")
    srafSideCell = layout.create_cell(f"{sraf_size}um_side_SRAF")
    srafEndCell = layout.create_cell(f"{sraf_size}um_end_SRAF")

    #Define line dimensions
    l_left = -size/2
    l_right = size/2
    l_bottom = (-cell_size/2)+(1.5*sraf_spacing)
    l_top = (cell_size/2)-(1.5*sraf_spacing)

    #Define sraf dimensions
    sraf_side_left = -sraf_size/2
    sraf_side_right = sraf_size/2
    sraf_side_top = l_top
    sraf_side_bottom = l_bottom
    sraf_end_left = l_left
    sraf_end_right = l_right
    sraf_end_bottom = -sraf_size/2
    sraf_end_top = sraf_size/2

    #Create the main feature shapes and insert it into their respective cells
    LineCell.shapes(l_line).insert(db.DBox(l_left, l_bottom, l_right, l_top))
    srafSideCell.shapes(l_line).insert(db.DBox(sraf_side_left, sraf_side_bottom, sraf_side_right, sraf_side_top))
    srafEndCell.shapes(l_line).insert(db.DBox(sraf_end_left, sraf_end_bottom, sraf_end_right, sraf_end_top))


    #Creates formatting regions
    CellBox = db.DBox((-cell_size/2),-cell_size/2,(cell_size/2),cell_size/2)
    BigBox = db.DBox(-cell_size,-cell_size,cell_size,cell_size)


#### Generate the Cell ####

    #Create sraf sub-instances
    if sraf_num>1:
         srafSideGroup = layout.create_cell("SRAF_side_instance")
         srafEndGroup = layout.create_cell("SRAF_end_instance")
         
         srafSideInst1a = db.DCellInstArray(srafSideCell,db.DTrans(db.DTrans.M0,-(size+sraf_step/2),0),db.DVector(-sraf_step,0),db.DVector(0,0),sraf_num,0)
         srafSideInst1b = db.DCellInstArray(srafSideCell,db.DTrans(db.DTrans.M90,(size+sraf_step/2),0),db.DVector(sraf_step,0),db.DVector(0,0),sraf_num,0)
         srafSideGroup.insert(srafSideInst1a)
         srafSideGroup.insert(srafSideInst1b)
         
         srafEndInst1a = db.DCellInstArray(srafEndCell,db.DTrans(db.DTrans.M0,0,l_top+(size/2)+(sraf_step/2)),db.DVector(0,sraf_step),db.DVector(0,0),sraf_num,0)
         srafEndInst1b = db.DCellInstArray(srafEndCell,db.DTrans(db.DTrans.M90,0,l_bottom-((sraf_step+size)/2)),db.DVector(0,-sraf_step),db.DVector(0,0),sraf_num,0)
         srafEndGroup.insert(srafEndInst1a)
         srafEndGroup.insert(srafEndInst1b)

    else:
         srafSideGroup = srafSideCell
         srafEndGroup = srafEndCell

    #Add SRAF instances to the LineCell
    srafSideToLine = db.DCellInstArray(srafSideGroup,db.DTrans(db.DTrans.M0,0,0))
    LineCell.insert(srafSideToLine)
    srafEndToLine = db.DCellInstArray(srafEndGroup,db.DTrans(db.DTrans.M0,0,0))
    LineCell.insert(srafEndToLine)

    #Generate the actual cell
    if iso:

        #Checks tone and adjusts if needed, then instances a single LineCell into the TopCell       
        if tone == "C":
            LineCell.flatten(-1,True)
            iso_region = db.Region(LineCell.shapes(l_line))
            clear_iso = db.Region(1000*CellBox) - iso_region
            clear_iso.break_(4,1)
            LineCell.clear_shapes()
            LineCell.shapes(l_line).insert(clear_iso)       
        ls_iso = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(ls_iso)

    else:
        #Create array
        LineArray1 = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0),db.DVector(pitch,0),db.DVector(0,0),math.ceil(num_lines/2),0)
        LineArray2 = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0),db.DVector(-pitch,0),db.DVector(0,0),math.ceil(num_lines/2),0)

        TopCell.insert(LineArray1)
        TopCell.insert(LineArray2)
        
        #Checks tone and adjusts if needed, then instances a single LineCell into the TopCell
        if tone == "C":
            TopCell.flatten(-1,True)
            dense_region = db.Region(TopCell.shapes(l_line))
            clear_dense = db.Region(CellBox*1000) - dense_region
            clear_dense.break_(4,1)
            TopCell.clear_shapes()
            TopCell.shapes(l_line).insert(clear_dense)


#No metro structure for this feature type


#### Angle transformations, sliver removal, and output ####
    
    #Apply angle rotation for all cells
    t = db.ICplxTrans(1,angle,0,0,0)
    [layout.cell(i).transform(t) for i in TopCell.each_child_cell()]

    #Clip a new cell that covers just the extents of the defined cell size, and eliminate subcells that contain slivers
    output_cell = layout.clip(TopCell,CellBox*1000)
    output_cell.name = (f"Line_w_SRAF_{tone}_{size}umSize_{pitch_type}_{angle}degrees_{sraf_num}SRAFs_{sraf_size}um_size_{sraf_step}um_apart")
    
    TopCell.prune_cell()

    #Export GDS
    layout.write("SRAF_Tester.gds")

    return output_cell

SRAF_cell()

Spiral PCell

In [11]:
# Import the needed extensions
import math
import klayout
import pya

#Going to make this a new PCell for Klayout to use, I essentially copied this from here: (https://www.youtube.com/watch?v=2e6DuFj0Xws)
class LSCell(pya.PCellDeclarationHelper):

    def __init__(self):
    
        #Initiallizes the super class
        super(LSCell, self).__init__()

        #Declare the parameters
        self.param("l", self.TypeLayer, "Layer", default = pya.LayerInfo(1,0)) #sets up the layer
        self.param("n", self.TypeInt, "Number of points", default = 64)
        self.param("inner_r", self.TypeDouble, "Inner Radius", default = 1) #defines starting point 
        self.param("outer_r", self.TypeDouble, "Outer Radius", default = 10) #defines the extent of the cell!
        self.param("width", self.TypeDouble, "Width", default = 1)
        self.param("spacing", self.TypeDouble, "Spacing", default = 1)
        self.param("inner_handle", self.TypeShape, "", default = pya.DPoint(0,0)) #handle used to change radius independent of radius parameter
        self.param("outer_handle", self.TypeShape, "", default = pya.DPoint(0,0)) 

        #Additional hidden parameters for whether the radius has changed or the "s" handle has been moved
        self.param("inner_memory", self.TypeDouble, "Inner Memory", default = 0.0, hidden = True)
        self.param("outer_memory", self.TypeDouble, "Outer Memory", default = 0.0, hidden = True)

    def display_text_impl(self):
        #Provides a descriptive text for the cell
        return "Spiral(L = " + str(self.l) + ", Outer radius = " + ('%.3f' % self.outer_r) + ")"
    
    def coerce_parameters_impl(self):

        #We employ coerce_parameters_impl to decide whether the handle or the numeric paramter has changed
        #(by comparing against the effective radius ru)
        #and set ru to the effective radius. We also update the numerical value or the shape,
        #depending on which one has not changed

        inner_handle_radius = None
        outer_handle_radius = None

        if isinstance(self.inner_handle, pya.DPoint):
            #if inner radius manually changed on layout by user, calculates value, computes distance in microns from 0,0 point
            inner_handle_radius = self.inner_handle.distance(pya.DPoint(0,0))
        if isinstance(self.outer_handle, pya.DPoint):
            #if inner radius manually changed on layout by user, calculates value, computes distance in microns from 0,0 point
            inner_handle_radius = self.outer_handle.distance(pya.DPoint(0,0))
        
        if abs(self.inner_r - self.inner_memory) < 1e-6 and abs(self.outer_r - self.outer_memory) < 1e-6: #case where either handle changes
            self.inner_memory = inner_handle_radius
            self.inner_r = inner_handle_radius
            self.outer_memory = outer_handle_radius
            self.outer_r = outer_handle_radius
        else: #case where the parameter changes
            self.inner_memory = self.inner_r
            self.inner_handle = pya.DPoint(-self.inner_r,0)
            self.outer_memory = self.outer_r
            self.outer_handle = pya.DPoint(-self.outer_r,0)
        
        #n must be larger than or equal to 4
        if self.n <= 4:
            self.n = 4
    
    def can_create_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: we can use any shape which has a finite bounding box
        return self.shape.is_box() or self.shape.is_polygon() or self.shape.is_path()
    
    def parameters_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: we set the inner and outer radii based on the bounding box of the shape being used
        self.inner_r = self.shape.bbox().width() * self.layout.dbu / 4
        self.outer_r = self.shape.bbox().width() * self.layout.dbu / 2
        self.l = self.layout.get_info(self.layer) #puts the new spiral shape on the same layer as the shape being converted
    
    def transformation_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: places the spiral in the same position as the converted shape
        return pya.Trans(self.shape.bbox().center())
    
    def produce_impl(self):
        #This is the main part of the function, where the shape is actually formed/created

        #Fetch the parameters, and convert to the database units being used in the layout
        inner_r_dbu = self.inner_r / self.layout.dbu
        outer_r_dbu = self.outer_r / self.layout.dbu

        #Set up array for the points to be placed for the spiral
        pts = []
        #The change in angle (in radians) being stepped across to drive the spiral
        da = math.pi * 2 / self.n
        #The change in radius being stepped across to expand the spiral.
        #After one rotation, want the spiral to have expanded by one width and one spacing.
        dr = (self.width+self.spacing)/self.n/self.layout.dbu

        #Initial conditions for spiral loop.
        current_radius = inner_r_dbu
        current_angle = 0

        #While loop used to fill out the spiral
        while current_radius < outer_r_dbu:
            #Adds a new point to the pts array for each while loop pass
            #Cosine and sine calculations converting the angle in radians to x,y coordinates
            pts.append(pya.Point.from_dpoint(pya.Dpoint(current_radius * math.cos(current_angle), current_radius * math.sin(current_angle))))
            #Steps the radius up
            current_radius += dr
            #Steps up the current angle.
            #Modulus of 2pi is being used to avoid angle buildup after 2pi is reached, reverting back to 0 instead
            current_angle = (current_angle+da)%(math.pi*2)
        
        #Creates the shape
        self.cell.shapes(self.l_layer).insert(pya.Path(pts, self.width/self.layout.dbu))

class MyLib(pya.Library):
    #The library where we put the PCell into
    def __init__(self):
        
        #Set the description
        self.description = "My Library"

        #Create the PCell declarations
        self.layout().register_pcell("Spiral", LSCell())

        #Register with the nam "MyLib"
        self.register("MyLib")

#Instantiate and register the library
MyLib()